In [ ]:
import requests
import os
from pprint import pprint
import json
import re
import re
import nltk
from stop_words import get_stop_words
from nltk.corpus import stopwords

In [ ]:
apikey = 'Your_API_Key'
query_url = f"https://api.nytimes.com/svc/archive/v1/2020/6.json?api-key={apikey}"
r = requests.get(query_url)
data = r.json()

In [ ]:
stop_words = list(get_stop_words('en'))         
nltk_words = list(stopwords.words('english'))   
stop_words.extend(nltk_words)


years = ['2020', '2021']
keyword_dict = {}
months = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']


for year in years:
    for month in months:
        query_url = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={apikey}"
        r = requests.get(query_url)
        data = r.json()
        articles = data["response"]["docs"]
        keyword_dict["year{y}month{m}".format(y=year, m=month)] = [set() for index in range(31)]
        for article in articles:
            if article['section_name'] == 'Business Day':
                corresponding_set = int(article['pub_date'][8:10])-1
                for word in article['keywords']:
                    re_word = re.sub('[^a-zA-Z0-9-. ]', '', word['value'].lower())
                    tokens = nltk.word_tokenize(re_word)
                    for token in tokens:
                        if not token in stop_words:
                            keyword_dict["year{y}month{m}".format(y=year, m=month)][corresponding_set].add(re.sub('\-$', '', token))
            
            else:
                continue

In [ ]:
with open('NYT2020_2021_keywords.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)